# Construct metadata from existing

* Metadata won't have the structure unless it's been filled out first. Starting from scratch means there are certain key-value pairs missing.
* Use 
* Start with a copy of an existing metadata to be template?

In [1]:
#import xml.etree.ElementTree as ET
#import json
import xmltodict

import metadata_update

In [2]:
XML_FILE = './metadata_xml/ca_hq_transit_areas.xml'
NEW_XML_FILE = './metadata_xml/ca_transit_routes.xml'

hqta = metadata_update.xml_to_json(XML_FILE)
routes = metadata_update.xml_to_json(NEW_XML_FILE)

Loading XML as JSON from ./metadata_xml/ca_hq_transit_areas.xml
Loading XML as JSON from ./metadata_xml/ca_transit_routes.xml


## Lift necessary stuff from 1st time through shp to file gdb

In [54]:
def lift_necessary_dataset_elements(metadata_json):
    m = metadata_json["metadata"]
    
    # Store this info in a dictionary
    d = {}
    
    # Bounding box
    d["idinfo"] = {}
    d["idinfo"]["spdom"] = m["idinfo"]["spdom"] 
    
    # Some description about geospatial layer
    d["spdoinfo"] = m["spdoinfo"] 
    
    # Spatial reference info
    d["spref"] = m["spref"] 
    
    # Field and entity attributes
    d["eainfo"] = m["eainfo"]
    
    return d

In [ ]:
route_info = lift_necessary_dataset_elements(routes)

In [58]:
def overwrite_default_with_dataset_elements(metadata_json):
    DEFAULT_XML = "./metadata_xml/default.xml"
    default_template = metadata_update.xml_to_json(DEFAULT_XML)
    default = default_template["metadata"]
    
    # Grab the necessary elements from my dataset
    necessary_elements = lift_necessary_dataset_elements(metadata_json)
    
    # Overwrite it in the default template
    for key, value in default.items():
        if key in necessary_elements.keys() and key != "idinfo":
            default[key] = necessary_elements[key]
        
        elif key == "idinfo":
            for k, v in value.items():
                if k == "spdom":
                    default[key][k] = necessary_elements[key][k]
           
            
    # Return the default template, but now with our dataset's info populated
    return default

In [61]:
routes_with_template = overwrite_default_with_dataset_elements(routes)

Loading XML as JSON from ./metadata_xml/default.xml


In [11]:
# Check if edition is available
# If yes, add 1 to it
# If not, set it to 1

hqta["metadata"]["idinfo"]

# https://blog.finxter.com/how-to-update-a-key-in-a-dictionary-in-python-if-the-key-doesnt-exist/
#device.setdefault('year',2019)


{'citation': {'citeinfo': {'title': 'my_dataset',
   'edition': '1',
   'geoform': 'vector digital data',
   'pubinfo': {'publish': 'California Integrated Travel Project'}}},
 'descript': {'abstract': 'Public. EPSG: 3310',
  'purpose': 'Summary sentence about dataset.'},
 'timeperd': {'timeinfo': {'rngdates': {'begdate': 'YYYYMMDD',
    'enddate': 'YYYYMMDD'}},
  'current': 'California'},
 'status': {'progress': 'Complete', 'update': 'Monthly'},
 'spdom': {'bounding': {'westbc': '-124.423914',
   'eastbc': '-113.465232',
   'northbc': '45.362934',
   'southbc': '32.441061'}},
 'keywords': [],
 'accconst': 'None',
 'useconst': 'None',
 'ptcontac': {'cntinfo': {'cntorgp': {'cntorg': 'Caltrans',
    'cntper': 'Eric Dasmalchi'},
   'cntpos': 'California Integrated Travel Project',
   'cntemail': 'eric.dasmalchi@dot.ca.gov'}},
 'native': 'Version 6.2 (Build 9200) ; Esri ArcGIS 10.8.1.14362'}

In [9]:
# Where is citation 
# Probably check whether that part exists, and if it doesn't, construct it?
# Or, it's easier to just grab the necessary sections and 
# replace the entirety 
hqta["metadata"]["eainfo"]

{'detailed': {'enttyp': {'enttypl': 'ca_hq_transit_areas'},
  'attr': [{'attrlabl': 'agency_n_1'},
   {'attrlabl': 'Shape_Length',
    'attrdef': 'Length of feature in internal units.',
    'attrdefs': 'Esri',
    'attrdomv': {'udom': 'Positive real numbers that are automatically generated.'}},
   {'attrlabl': 'calitp_i_1'},
   {'attrlabl': 'OBJECTID',
    'attrdef': 'Internal feature number.',
    'attrdefs': 'Esri',
    'attrdomv': {'udom': 'Sequential unique whole numbers that are automatically generated.'}},
   {'attrlabl': 'Shape_Area',
    'attrdef': 'Area of feature in internal units squared.',
    'attrdefs': 'Esri',
    'attrdomv': {'udom': 'Positive real numbers that are automatically generated.'}},
   {'attrlabl': 'hqta_type'},
   {'attrlabl': 'calitp_itp'},
   {'attrlabl': 'agency_nam'},
   {'attrlabl': 'Shape',
    'attrdef': 'Feature geometry.',
    'attrdefs': 'Esri',
    'attrdomv': {'udom': 'Coordinates defining the features.'}}]}}

In [6]:
SAMPLE_DATASET_INFO = {
    "dataset_name": "my_dataset", 
    "publish_entity": "California Integrated Travel Project", 

    "abstract": "Public. EPSG: 3310",
    "purpose": "Summary sentence about dataset.", 

    "beginning_date": "YYYYMMDD",
    "end_date": "YYYYMMDD",
    "place": "California",

    "status": "Complete", 
    "frequency": "Monthly",
    
    "theme_topics": [], 

    "methodology": "Detailed methodology description", 
    
    "data_dict_type": "CSV",
    "data_dict_url": "some_url", 

    "contact_organization": "Caltrans", 
    "contact_person": "Analyst Name", 
    "contact_email": "hello@calitp.org" 
}


# Overwrite the metadata after dictionary of dataset info is supplied
def overwrite_metadata_json(metadata_json, DATASET_INFO):
    d = DATASET_INFO
    new_metadata = metadata_json.copy()
    m = new_metadata["metadata"]
    
    m["idinfo"]["citation"]["citeinfo"]["title"] = d["dataset_name"]
    m["idinfo"]["citation"]["citeinfo"]["pubinfo"]["publish"] = d["publish_entity"]
    
    m["idinfo"]["descript"]["abstract"] = d["abstract"]
    m["idinfo"]["descript"]["purpose"] = d["purpose"]
    
    m["idinfo"]["timeperd"]["timeinfo"]["rngdates"]["begdate"] = d["beginning_date"]
    m["idinfo"]["timeperd"]["timeinfo"]["rngdates"]["enddate"] = d["end_date"]
    m["idinfo"]["timeperd"]["current"] = d["place"]
    
    m["idinfo"]["status"]["progress"] = d["status"]
    m["idinfo"]["status"]["update"] = d["frequency"]

    m["idinfo"]["keywords"] = d["theme_topics"]    

    m["dataqual"]["lineage"]["procstep"]["procdesc"] = d["methodology"]    
    
    m["eainfo"]["detailed"][0]["enttyp"]["enttypl"] = d["dataset_name"]    
    m["eainfo"]["detailed"][1]["enttyp"]["enttypd"] = d["data_dict_type"]    
    m["eainfo"]["detailed"][1]["enttyp"]["enttypds"] = d["data_dict_url"]    
  
    m["metainfo"]["metc"]["cntinfo"]["cntorgp"]["cntorg"] = d["contact_organization"]    
    m["metainfo"]["metc"]["cntinfo"]["cntorgp"]["cntper"] = d["contact_person"]    
    m["metainfo"]["metc"]["cntinfo"]["cntpos"] = d["publish_entity"]    
    m["metainfo"]["metc"]["cntinfo"]["cntemail"] = d["contact_email"]    

    return new_metadata 


In [7]:
new_metadata = overwrite_metadata_json(hqta, SAMPLE_DATASET_INFO)


KeyError: 0

{'metadata': {'idinfo': {'citation': {'citeinfo': {'title': 'ca_transit_routes',
     'geoform': 'vector digital data'}},
   'spdom': {'bounding': {'westbc': '-124.226362',
     'eastbc': '-69.965466',
     'northbc': '48.716492',
     'southbc': '25.849554'}},
   'accconst': 'None',
   'useconst': 'None',
   'native': 'Version 6.2 (Build 9200) ; Esri ArcGIS 10.8.1.14362'},
  'spdoinfo': {'direct': 'Vector',
   'ptvctinf': {'sdtsterm': {'sdtstype': 'String', 'ptvctcnt': '7895'}}},
  'spref': {'horizsys': {'geograph': {'latres': '8.983152841195215e-09',
     'longres': '8.983152841195215e-09',
     'geogunit': 'Decimal Degrees'},
    'geodetic': {'horizdn': 'D WGS 1984',
     'ellips': 'WGS 1984',
     'semiaxis': '6378137.0',
     'denflat': '298.257223563'}}},
  'eainfo': {'detailed': {'enttyp': {'enttypl': 'ca_transit_routes'},
    'attr': [{'attrlabl': 'OBJECTID',
      'attrdef': 'Internal feature number.',
      'attrdefs': 'Esri',
      'attrdomv': {'udom': 'Sequential unique who